In [1]:
import numpy as np
import tensorflow as tf
import gzip
import pickle
import sys
import copy
import os.path
import pdb
import re
from ddm.run_split import SplitMnistGenerator
from ddm.alg.cla_models_multihead import MFVI_IBP_NN, Vanilla_NN
from ddm.alg.utils import get_scores, concatenate_results
from ddm.alg.vcl import run_vcl
from copy import deepcopy

from bayes_opt import BayesianOptimization

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

W0816 18:17:56.460163 140197656663872 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:9: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0816 18:17:56.461135 140197656663872 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:13: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.



# IBP network

In [2]:
hidden_size = [100]
batch_size = 128
no_epochs = 100
alpha0 = 1.0
tau0=1.0 # initial temperature
ANNEAL_RATE=0.000
MIN_TEMP=0.1

# Run vanilla VCL
tf.set_random_seed(12)
np.random.seed(1)

ibp_acc = np.array([])

coreset_size = 0
val = True
data_gen = SplitMnistGenerator(val)
single_head=False
in_dim, out_dim = data_gen.get_dims()
x_testsets, y_testsets = [], []
x_valsets, y_valsets = [], []
for task_id in range(data_gen.max_iter):
    
    tf.reset_default_graph()
    if val:
        x_train, y_train, x_test, y_test, x_val, y_val = data_gen.next_task()
        x_valsets.append(x_val)
        y_valsets.append(y_val)
    else:    
        x_train, y_train, x_test, y_test = data_gen.next_task()
    x_testsets.append(x_test)
    y_testsets.append(y_test)

    # Set the readout head to train
    head = 0 if single_head else task_id
    bsize = x_train.shape[0] if (batch_size is None) else batch_size
    
    # Train network with maximum likelihood to initialize first model
    if task_id == 0:
        ml_model = Vanilla_NN(in_dim, hidden_size, out_dim, x_train.shape[0])
        ml_model.train(x_train, y_train, task_id, no_epochs, bsize)
        mf_weights = ml_model.get_weights()
        mf_variances = None
        mf_betas = None
        ml_model.close_session()

    # Train on non-coreset data
    mf_model = MFVI_IBP_NN(in_dim, hidden_size, out_dim, x_train.shape[0], prev_means=mf_weights, 
                           prev_log_variances=mf_variances, prev_betas=mf_betas,alpha0=alpha0,
                           learning_rate=0.01, temp=tau0, temp_prior=1.0, no_pred_samples=100)
    mf_model.train(x_train, y_train, head, no_epochs, bsize,
                   anneal_rate=ANNEAL_RATE, min_temp=MIN_TEMP)
    mf_weights, mf_variances, mf_betas = mf_model.get_weights()

    acc = get_scores(mf_model, x_valsets, y_valsets, single_head)
    ibp_acc = concatenate_results(acc, ibp_acc)
    
    mf_model.close_session()
    
ibp_acc

W0813 18:37:30.424512 140408376342336 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:53: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0813 18:37:30.427169 140408376342336 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:164: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0813 18:37:30.482261 140408376342336 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:58: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0813 18:37:30.588547 140408376342336 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:62: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

W0813 18:37:30.590138 140408376342336 deprecat

Epoch: 0001 cost= 0.059754695
Epoch: 0006 cost= 0.000913144
Epoch: 0011 cost= 0.000234285
Epoch: 0016 cost= 0.000082365
Epoch: 0021 cost= 0.000042895
Epoch: 0026 cost= 0.000023777
Epoch: 0031 cost= 0.000014887
Epoch: 0036 cost= 0.000009923
Epoch: 0041 cost= 0.000006549
Epoch: 0046 cost= 0.000004535
Epoch: 0051 cost= 0.000003333
Epoch: 0056 cost= 0.000002337
Epoch: 0061 cost= 0.000001674
Epoch: 0066 cost= 0.000001251
Epoch: 0071 cost= 0.000000906
Epoch: 0076 cost= 0.000000680
Epoch: 0081 cost= 0.000000512
Epoch: 0086 cost= 0.000000387
Epoch: 0091 cost= 0.000000281
Epoch: 0096 cost= 0.000000211


W0813 18:37:40.962682 140408376342336 deprecation.py:323] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:525: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


Optimization Finished!
z_discrete: (1, ?, 100)
biases: <unknown>
pre: <unknown>


W0813 18:37:41.177778 140408376342336 deprecation.py:323] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:496: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


...

W0813 18:37:41.663980 140408376342336 deprecation.py:323] From /home/skessler/anaconda2/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0813 18:37:42.680103 140408376342336 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:503: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

W0813 18:37:42.716534 140408376342336 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:621: The name tf.summary.scalar is deprecated. Please use tf.compat.v1.summary.scalar instead.

W0813 18:37:42.732589 140408376342336 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:630: The name tf

_Z: (1, ?, 100)


W0813 18:37:42.987454 140408376342336 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:913: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch: 0001 cost= 19.072743214
Epoch: 0006 cost= 5.168781454
Epoch: 0011 cost= 3.800158814
Epoch: 0016 cost= 3.674991938
Epoch: 0021 cost= 3.574922829
Epoch: 0026 cost= 3.511328907
Epoch: 0031 cost= 3.403818682
Epoch: 0036 cost= 3.414693139
Epoch: 0041 cost= 3.346116540
Epoch: 0046 cost= 3.362689755
Epoch: 0051 cost= 3.332217416
Epoch: 0056 cost= 3.280205180
Epoch: 0061 cost= 3.252052155
Epoch: 0066 cost= 3.273583114
Epoch: 0071 cost= 3.220419619
Epoch: 0076 cost= 3.239492937
Epoch: 0081 cost= 3.198088482
Epoch: 0086 cost= 3.186336327
Epoch: 0091 cost= 3.173645304
Epoch: 0096 cost= 3.183016286
Optimization Finished!
z_discrete: (1, ?, 100)
biases: <unknown>
pre: <unknown>
z_discrete: (1, ?, 100)
biases: <unknown>
pre: <unknown>
z_discrete: (1, ?, 100)
biases: <unknown>
pre: <unknown>
_Z: (1, ?, 100)
Epoch: 0001 cost= 1.909118086
Epoch: 0006 cost= 1.566231799
Epoch: 0011 cost= 1.535252309
Epoch: 0016 cost= 1.504906576
Epoch: 0021 cost= 1.506916551
Epoch: 0026 cost= 1.488153032
Epoch: 00

array([[0.9607565 ,        nan,        nan,        nan,        nan],
       [0.8141844 , 0.92654261,        nan,        nan,        nan],
       [0.6212766 , 0.87267385, 0.92902882,        nan,        nan],
       [0.62222222, 0.85651322, 0.90394877, 0.94561934,        nan],
       [0.46146572, 0.67825661, 0.91141942, 0.82678751, 0.9293999 ]])

In [ ]:
# Run Vanilla VCL
tf.reset_default_graph()
tf.set_random_seed(12)
np.random.seed(1)
hidden_size = [10]
coreset_size = 0

data_gen = SplitMnistGenerator()
vcl_result = run_vcl(hidden_size, no_epochs, data_gen, 
                              lambda a: a, coreset_size, batch_size, single_head)
print(vcl_result)

In [ ]:
# # Run IBP VCL
# tf.reset_default_graph()
# tf.set_random_seed(12)
# np.random.seed(1)
# coreset_size = 0

# hidden_size = [50]
# batch_size = 128
# no_epochs = 100
# alpha0 = 5.0
# tau0=0.1 # initial temperature
# temp_prior=1.0
# ANNEAL_RATE=0.000
# MIN_TEMP=0.1
# single_head=False

# # data_gen = SplitMnistGenerator()
# # vcl_ibp_result = vcl.run_vcl_ibp(hidden_size=hidden_size, no_epochs=no_epochs, data_gen=data_gen,
# #                                   batch_size=batch_size, single_head=single_head, alpha0=alpha0,
# #                                   learning_rate=0.01, temp_prior=temp_prior, no_pred_samples=100,
# #                                   tau0=tau0, tau_anneal_rate=ANNEAL_RATE, tau_min=MIN_TEMP)
# # print(vcl_ibp_result)

In [ ]:
_ibp_acc = np.nanmean(ibp_acc, 1)
_vcl_result = np.nanmean(vcl_result, 1)

In [ ]:
_vcl_result

In [ ]:
plt.rc('text', usetex=True)
#plt.rc('font', family='serif')

fig = plt.figure(figsize=(7,3))
ax = plt.gca()
plt.plot(np.arange(len(_ibp_acc))+1, _ibp_acc, label='VCL + IBP', marker='o')
plt.plot(np.arange(len(_vcl_result))+1, _vcl_result, label='VCL', marker='o')
ax.set_xticks(range(1, len(_ibp_acc)+1))
ax.set_ylabel('Average accuracy')
ax.set_xlabel('\# tasks')
ax.legend()
fig.show()

# Weight pruning

In [2]:
class MFVI_IBP_NN_prune(MFVI_IBP_NN):
    def __init__(self, input_size, hidden_size, output_size, training_size,
                 no_train_samples=10, no_pred_samples=100, prev_means=None, prev_log_variances=None,
                 prev_betas=None, learning_rate=0.001,
                 prior_mean=0, prior_var=1, alpha0=5., beta0=1., lambda_1=1., lambda_2=1.,
                 tensorboard_dir='logs', name='ibp_wp', min_temp=0.5, tb_logging=True):

        super(MFVI_IBP_NN_prune, self).__init__(input_size, hidden_size, output_size, training_size,
                 no_train_samples, no_pred_samples, prev_means, prev_log_variances,
                 prev_betas, learning_rate,
                 prior_mean, prior_var, alpha0, beta0, lambda_1, lambda_2,
                 tensorboard_dir, name, min_temp, tb_logging)


    def prune_weights(self, X_test, Y_test, task_id):
        """ Performs weight pruning
        Args:
            X_test: numpy array
            Y_test: numpy array
            task_id: int
        :return: cutoffs, accs via naive pruning, accs via snr pruning,
        weight values, sigma values of network
        """

        def reset_weights(pr_mus, pr_sigmas, _mus, _sigmas):
            """ Reset weights of graph to original values
            Args:
                pr_mus: list of tf variables which have been pruned
                pr_sigmas: list of tf variables which have been pruned
                _mus: list of cached mus in numpy
                _sigmas: list of cached sigmas in numpy
            """

            for v, _v in zip(pr_mus, _mus):
                self.sess.run(tf.assign(v, tf.cast(_v, v.dtype)))

            for v, _v in zip(pr_sigmas, _sigmas):
                self.sess.run(tf.assign(v, tf.cast(_v, v.dtype)))

        def pruning(remove_pct, weightvalues_ibp, weightvalues, sigmavalues, Zs, Z_tiled,
                    mu_weights, mu_sigmas, b_weights, b_sigmas, uncert_pruning=True):
            """ Performs weight pruning experiment
            Args:
                weightvalues_ibp: np array of weights with ibp mask applied
                weightvalues: np array of weights
                sigmavalues: np array of sigmas
                Zs: list of tf IBP masks
                Z_tiled: list of tf IBP masks, tiled for application to weights, not biases
                mu_weights: list of tf weight variable
                mu_sigmas: list of tf sigma variables
                b_weights: list of tf mean variables for biases
                b_sigmas: list of tf sigma variables for biases
                uncert_pruning: bool pruning by snr
            """
            if uncert_pruning:
                sorted_STN = np.sort(np.abs(weightvalues_ibp) / sigmavalues)
            else:
                sorted_STN = np.sort(np.abs(weightvalues))
            cutoff = sorted_STN[int(remove_pct * len(sorted_STN))]
            
            # Weights
            for v, s in zip(mu_weights, mu_sigmas):
                if uncert_pruning:
                    # TODO apply Z mask
                    snr = tf.abs(v) / tf.exp(0.5*s)
                    mask = tf.greater_equal(snr, cutoff)
                else:
                    mask = tf.greater_equal(tf.abs(v), cutoff)
                self.sess.run(tf.assign(v, tf.multiply(v, tf.cast(mask, v.dtype))))
                self.sess.run(tf.assign(s, tf.multiply(s, tf.cast(mask, s.dtype))))  # also apply zero std to weight!!!
            
            # Biases
            for v, s in zip(b_weights, b_sigmas):
                if uncert_pruning:
                    # TODO apply Z mask
                    snr = tf.abs(v) / tf.exp(0.5*s)
                    mask = tf.greater_equal(snr, cutoff)
                else:
                    mask = tf.greater_equal(tf.abs(v), cutoff)
                self.sess.run(tf.assign(v, tf.multiply(v, tf.cast(mask, v.dtype))))
                self.sess.run(tf.assign(s, tf.multiply(s, tf.cast(mask, s.dtype))))  # also apply zero std to weight!!!
            
            accs = []
            for _ in range(10):
                accs.append(self.sess.run(self.acc, {self.x: X_test,
                                                     self.y: Y_test,
                                                     self.task_idx: task_id,
                                                     self.temp: self.lambda_1,
                                                     self.training: False}))
            print("%.2f, %s" % (np.sum(sorted_STN < cutoff) / len(sorted_STN), np.mean(accs)))
            return np.mean(accs)

        # get weights
        weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=tf.get_variable_scope().name)
        
        # Get weights from network
        mus_w = []
        mus_b = []
        sigmas_w = []
        sigmas_b = []
        for v in weights:
            if re.match("^([w])(_mu_)([0-9]+)(:0)$", v.name):
                mus_w.append(v)
            elif re.match("^([w])(_sigma_)([0-9]+)(:0)$", v.name):
                sigmas_w.append(v)
            elif re.match("^([b])(_mu_)([0-9]+)(:0)$", v.name):
                mus_b.append(v)
            elif re.match("^([b])(_sigma_)([0-9]+)(:0)$", v.name):
                sigmas_b.append(v)
            else:
                print("Un-matched: {}".format(v.name))
        # Get mask
        # Need to apply mask to weights and biases for pruning
        # Do not apply to head weights though!
        # Zs \in [x_test.size, dout], just need one sample of Z!
        Zs = self.sess.run(self.Z, {self.x: X_test,
                                    self.y: Y_test,
                                    self.task_idx: task_id,
                                    self.temp: self.lambda_1,
                                    self.training: False})[0] # z mask for each layer in a list, each Z \in dout

        # cache
        _mus_w = [self.sess.run(w) for w in mus_w]
        _sigmas_w = [self.sess.run(w) for w in sigmas_w]
        _mus_b = [self.sess.run(w) for w in mus_b]
        _sigmas_b = [self.sess.run(w) for w in sigmas_b]

        # flatten values for cut-off finding
        # Tile Zs [dout] --> [din, dout]
        dins = [x.shape[0] for x in _mus_w]
        print("dins: {}".format(len(dins)))
        print("_mus: {}".format(len(_mus_w)))
        Z_tiled = []
        t_Z_tiled = []
        for i in range(len(dins)):
            Z_tiled.append(np.tile(Zs[i][0, :], (dins[i], 1))) 
            t_Z_tiled.append(tf.tile(tf.squeeze(self.Z[i], axis=0), [dins[i], 1]))
        
        # multiply Zs with 
        xx = []
        yy = []
        for i in range(len(mus_w)):
            xx.append(np.multiply(self.sess.run(mus_w[i]), Z_tiled[i]).flatten())
            yy.append(np.multiply(self.sess.run(mus_b[i]), Zs[i]).flatten())
            
        weightvalues_ibp = np.hstack(np.array(xx + yy))
        weightvalues = np.hstack(np.array([self.sess.run(w).flatten() for w in mus_w + mus_b]))
        sigmavalues_tr = np.hstack(np.array([self.sess.run(tf.exp(0.5*s)).flatten() for s in sigmas_w + sigmas_b]))
    
        xs = np.append(0.05 * np.array(range(20)), np.array([0.98, 0.99, 0.999]))
        ya = []
        for pct in xs:
            ya.append(pruning(pct, weightvalues_ibp, weightvalues, sigmavalues_tr, self.Z, t_Z_tiled,
                              mus_w, sigmas_w, mus_b, sigmas_b, uncert_pruning=False))

        # reset etc.
        reset_weights(mus_w, sigmas_w, _mus_w, _sigmas_w)
        reset_weights(mus_b, sigmas_b, _mus_b, _sigmas_b)
        pdb.set_trace()
        yb = []
        for pct in xs:
            yb.append(pruning(pct, weightvalues_ibp, weightvalues, sigmavalues_tr, self.Z, t_Z_tiled,
                              mus_w, sigmas_w, mus_b, sigmas_b, uncert_pruning=True))
            
        reset_weights(mus_w, sigmas_w, _mus_w, _sigmas_w)
        reset_weights(mus_b, sigmas_b, _mus_b, _sigmas_b)
        
        return xs, ya, yb, weightvalues, sigmavalues_tr

In [ ]:
hidden_size = [100]
batch_size = 128
no_epochs = 10
alpha0 = 1.0
tau0=1.0 # initial temperature
ANNEAL_RATE=0.000
MIN_TEMP=0.1

tf.set_random_seed(12)
np.random.seed(1)

ibp_acc = np.array([])

coreset_size = 0
val = False
data_gen = SplitMnistGenerator(val, num_tasks=1)
single_head=False
in_dim, out_dim = data_gen.get_dims()
x_testsets, y_testsets = [], []
x_valsets, y_valsets = [], []
for task_id in range(data_gen.max_iter):
    
    tf.reset_default_graph()
    if val:
        x_train, y_train, x_test, y_test, x_val, y_val = data_gen.next_task()
        x_valsets.append(x_val)
        y_valsets.append(y_val)
    else:    
        x_train, y_train, x_test, y_test = data_gen.next_task()
    x_testsets.append(x_test)
    y_testsets.append(y_test)

    # Set the readout head to train
    head = 0 if single_head else task_id
    bsize = x_train.shape[0] if (batch_size is None) else batch_size
    
    # Train network with maximum likelihood to initialize first model
    if task_id == 0:
        ml_model = Vanilla_NN(in_dim, hidden_size, out_dim, x_train.shape[0])
        ml_model.train(x_train, y_train, task_id, 100, bsize)
        mf_weights = ml_model.get_weights()
        mf_variances = None
        mf_betas = None
        ml_model.close_session()

    # Train on non-coreset data
    mf_model = MFVI_IBP_NN_prune(in_dim, hidden_size, out_dim, x_train.shape[0], prev_means=mf_weights, 
                           prev_log_variances=mf_variances, prev_betas=mf_betas,alpha0=alpha0,
                           learning_rate=0.01, lambda_1=tau0, lambda_2=1.0, no_pred_samples=100)
    mf_model.train(x_train, y_train, head, no_epochs, bsize,
                   anneal_rate=ANNEAL_RATE, min_temp=MIN_TEMP)
    
    xs, ya, yb, _, _  = mf_model.prune_weights(x_test, y_test, head)
    
    mf_model.close_session()

W0816 18:17:57.806393 140197656663872 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:56: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0816 18:17:57.811012 140197656663872 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:167: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0816 18:17:57.882209 140197656663872 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:61: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0816 18:17:57.986377 140197656663872 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:65: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.



Epoch: 0001 cost= 0.047497832
Epoch: 0006 cost= 0.000949820
Epoch: 0011 cost= 0.000255335
Epoch: 0016 cost= 0.000100076
Epoch: 0021 cost= 0.000052391
Epoch: 0026 cost= 0.000029428
Epoch: 0031 cost= 0.000018897
Epoch: 0036 cost= 0.000012422
Epoch: 0041 cost= 0.000008246
Epoch: 0046 cost= 0.000005765
Epoch: 0051 cost= 0.000004355
Epoch: 0056 cost= 0.000002994
Epoch: 0061 cost= 0.000002143
Epoch: 0066 cost= 0.000001619
Epoch: 0071 cost= 0.000001182
Epoch: 0076 cost= 0.000000894
Epoch: 0081 cost= 0.000000666
Epoch: 0086 cost= 0.000000527
Epoch: 0091 cost= 0.000000367
Epoch: 0096 cost= 0.000000278


W0816 18:18:12.404314 140197656663872 deprecation.py:323] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:534: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


Optimization Finished!
z_discrete: (1, ?, 100)
biases: <unknown>
pre: <unknown>


W0816 18:18:12.944829 140197656663872 deprecation.py:323] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:505: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


z_discrete: (1, ?, 100)
biases: <unknown>
pre: <unknown>
z_discrete: (1, ?, 100)
biases: <unknown>
pre: <unknown>


W0816 18:18:13.505430 140197656663872 deprecation.py:323] From /home/skessler/anaconda2/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0816 18:18:14.818208 140197656663872 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:512: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

W0816 18:18:14.865417 140197656663872 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:630: The name tf.summary.scalar is deprecated. Please use tf.compat.v1.summary.scalar instead.

W0816 18:18:14.889670 140197656663872 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:639: The name tf

_Z: (1, ?, 100)


W0816 18:18:15.136775 140197656663872 deprecation_wrapper.py:119] From /home/skessler/Projects/IBP_BNN/ddm/alg/cla_models_multihead.py:921: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch: 0001 train cost= 19.118880744
Epoch: 0006 train cost= 5.136846292
Optimization Finished!
Un-matched: w_0:0
Un-matched: b_0:0
Un-matched: w_h_0:0
Un-matched: b_h_0:0
Un-matched: beta_a_0:0
Un-matched: beta_b_0:0
Un-matched: w_mu_h_0:0
Un-matched: b_mu_h_0:0
Un-matched: w_sigma_h_0:0
Un-matched: b_sigma_h_0:0
dins: 1
_mus: 1
0.00, 0.9973711
0.05, 0.9970071
0.10, 0.9971911
0.15, 0.99726856
0.20, 0.99732906
0.25, 0.9969385
0.30, 0.997244
0.35, 0.9972345
0.40, 0.9970709
0.45, 0.9973839
0.50, 0.9971201
0.55, 0.99734706
0.60, 0.9973661
0.65, 0.99765337
0.70, 0.9973229
0.75, 0.9972355
0.80, 0.9971466
0.85, 0.9971503
0.90, 0.9968709
0.95, 0.99729174
0.98, 0.9965154
0.99, 0.9927977
1.00, 0.7786955
> <ipython-input-2-1dd24ffc7e9a>(160)prune_weights()
-> yb = []
(Pdb) xx[0].shape
(78400,)
(Pdb) yy[0].shape
(78400,)
(Pdb) self.sess.run(mus_b[0])
array([ 9.53524411e-01, -7.23002732e-01,  6.56470120e-01, -3.89161527e-01,
        5.11614263e-01,  2.02938989e-01, -2.04024628e-01, -8.99157077e-02

In [ ]:
plt.plot(xs, ya, label='$|\mu|$')
plt.plot(xs, yb, label='$ \\frac{|\mu|}{\sigma}$')
plt.xlabel('cut-off')
plt.ylabel('acc')
plt.legend()
plt.show()